# BERT

### bert_en_CASED_L-12_H-768_A-12

In [1]:
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import re
import numpy as np

input_file = 'test_data-CLEANED.csv' #'train_data-CLEANED.csv'
output_file = 'test_data-CLEANED-0.csv' #'train_data-CLEANED-0.csv'

In [2]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "bert_en_cased_preprocess_3")
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer(
    "bert_en_cased_L-12_H-768_A-12_4",
    trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].


In [3]:
# WARNING:tensorflow:Please fix your imports. Module tensorflow.python.training.tracking.data_structures has been moved to tensorflow.python.trackable.data_structures. The old module will be deleted in version 2.11.
# probably no need to worry about this warning

In [4]:
embedding_model = tf.keras.Model(text_input, pooled_output)

In [5]:
# embeddings can be created for both words and sentences
sentences = tf.constant(["Hello World", "Hello"])
embeddings = embedding_model(sentences)
print(embeddings.shape)
print(embeddings[0])

(2, 768)
tf.Tensor(
[-7.53340602e-01  5.49169838e-01  9.99968171e-01 -9.96426284e-01
  9.68468726e-01  8.54189992e-01  9.88956332e-01 -9.67850387e-01
 -9.90116835e-01 -7.24072158e-01  9.92024839e-01  9.99358296e-01
 -9.94596243e-01 -9.99923229e-01  6.62646115e-01 -9.87569571e-01
  9.93816197e-01 -6.79853141e-01 -9.99992788e-01 -4.27096814e-01
 -3.48717332e-01 -9.99967933e-01  3.19312394e-01  9.30487752e-01
  9.85215783e-01  1.42147362e-01  9.95145082e-01  9.99988794e-01
  9.28473175e-01  9.11127850e-02  2.59872586e-01 -9.95870948e-01
  7.95685530e-01 -9.99630809e-01  3.37901413e-01 -2.84113642e-02
  6.71541274e-01 -4.58086342e-01  7.65442669e-01 -8.84961665e-01
 -8.60471189e-01 -4.38804179e-01  5.23509562e-01 -6.69429839e-01
  9.23713684e-01  5.85835278e-01  6.16375171e-02 -3.06763221e-02
 -2.70662904e-01  9.99977708e-01 -9.82207298e-01  9.99949753e-01
 -9.79321420e-01  9.99498248e-01  9.96955991e-01  5.84713697e-01
  9.96896327e-01  1.93566784e-01 -9.95517135e-01  4.97742742e-01
  9.8

### Embeddings for parts of affiliations

In [6]:
t_data = pd.read_csv(input_file, sep = ";")
#t_data.drop(columns = ['Unnamed: 0'], inplace = True)
t_data

,part,labels,affil_index
0,Aventis Pharma,Institution,0
1,Romainville,City,0
2,France. christophe.dini@aventis.com,Country,0
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1
4,China Pharmaceutical University,Institution,1
...,...,...,...
311,People's Republic of China.,Country,58
312,National Center for Natural Products Research,Institution,59
313,The University of Mississippi,Institution,59
314,MS 38677,State,59


In [7]:
parts = t_data.iloc[:, 0].tolist()
parts

['Aventis Pharma',
 'Romainville',
 'France. christophe.dini@aventis.com',
 'Jiangsu Key Laboratory of Drug Design and Optimization',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China',
 'Key Laboratory on Protein Chemistry and Structural Biology',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China',
 'China Pharmaceutical University',
 'Nanjing 210009',
 'China.',
 'Universidade Federal do Espírito Santo',
 'Rodovia BR 101 Norte',
 'Km 60',
 '29932-900',
 'Brazil.',
 'Jamia Hamdard',
 'New Delhi 110062',
 'India.',
 'Pondicherry University',
 'Puducherry 605 014',
 'India. Electronic address: tchinnasamy@hotmail.com.',
 'Payame Noor University',
 'Khoy 58168-45164',
 'Iran. saghatforoush@gmail.com',
 'Jadavpur University',
 'Kolkata 700 032',
 'India.',
 'Jagiellonian University Medical College',
 'Kraków',
 'Poland. Electronic address: mfmalaws@cyf-kr.edu.pl.',
 'Istituto Superiore di Sanità',
 'Viale Regina Elena 299',
 '00161 Roma',
 'Italy. carlo.mustazza@

In [8]:
#embeddings = embedding_model(tf.constant(parts))
#print(embeddings.shape)

In [9]:
embeddings_parts = []
for part in parts:
    try:
        embeddings_parts.append(embedding_model(tf.constant([part])).numpy().tolist())
    except:
        print(part)
        embeddings_parts.append("X")
t_data["embeddings_csd-BERT"] = embeddings_parts
#took 13 mins to run for 3000 parts of 600 affiliations

In [10]:
t_data

,part,labels,affil_index,embeddings_csd-BERT
0,Aventis Pharma,Institution,0,"[[-0.6225269436836243, 0.41175007820129395, 0...."
1,Romainville,City,0,"[[-0.7443768978118896, 0.37689119577407837, 0...."
2,France. christophe.dini@aventis.com,Country,0,"[[-0.7713785767555237, 0.44986480474472046, 0...."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[[-0.615035891532898, 0.31155872344970703, 0.9..."
4,China Pharmaceutical University,Institution,1,"[[-0.738187313079834, 0.46809735894203186, 0.9..."
...,...,...,...,...
311,People's Republic of China.,Country,58,"[[-0.7836729288101196, 0.5700669884681702, 0.9..."
312,National Center for Natural Products Research,Institution,59,"[[-0.6381785273551941, 0.46710556745529175, 0...."
313,The University of Mississippi,Institution,59,"[[-0.676986813545227, 0.4199213683605194, 0.99..."
314,MS 38677,State,59,"[[-0.7675344944000244, 0.5155598521232605, 0.9..."


In [11]:
t_data["embeddings_csd-BERT"] = t_data["embeddings_csd-BERT"].apply(lambda x: x[0])
t_data

,part,labels,affil_index,embeddings_csd-BERT
0,Aventis Pharma,Institution,0,"[-0.6225269436836243, 0.41175007820129395, 0.9..."
1,Romainville,City,0,"[-0.7443768978118896, 0.37689119577407837, 0.9..."
2,France. christophe.dini@aventis.com,Country,0,"[-0.7713785767555237, 0.44986480474472046, 0.9..."
3,Jiangsu Key Laboratory of Drug Design and Opti...,Institution,1,"[-0.615035891532898, 0.31155872344970703, 0.99..."
4,China Pharmaceutical University,Institution,1,"[-0.738187313079834, 0.46809735894203186, 0.99..."
...,...,...,...,...
311,People's Republic of China.,Country,58,"[-0.7836729288101196, 0.5700669884681702, 0.99..."
312,National Center for Natural Products Research,Institution,59,"[-0.6381785273551941, 0.46710556745529175, 0.9..."
313,The University of Mississippi,Institution,59,"[-0.676986813545227, 0.4199213683605194, 0.999..."
314,MS 38677,State,59,"[-0.7675344944000244, 0.5155598521232605, 0.99..."


In [12]:
t_data.loc[0, 'embeddings_csd-BERT']

[-0.6225269436836243,
 0.41175007820129395,
 0.999845027923584,
 -0.9915947318077087,
 0.9590234756469727,
 0.9239410161972046,
 0.945000946521759,
 -0.9926048517227173,
 -0.9682585597038269,
 -0.5781539082527161,
 0.9827021956443787,
 0.9979745149612427,
 -0.9982824921607971,
 -0.9997150897979736,
 0.6656962037086487,
 -0.951337993144989,
 0.9868899583816528,
 -0.5472780466079712,
 -0.9999634027481079,
 -0.5011404156684875,
 -0.4892568290233612,
 -0.9998661875724792,
 0.18888989090919495,
 0.969135582447052,
 0.9421300888061523,
 0.07583226263523102,
 0.9812483191490173,
 0.9999623894691467,
 0.6785068511962891,
 -0.030462410300970078,
 0.1234230101108551,
 -0.9912542700767517,
 0.9069017171859741,
 -0.9993001818656921,
 0.2261529266834259,
 0.4521808922290802,
 0.75091552734375,
 -0.232376366853714,
 0.7489871382713318,
 -0.9633144736289978,
 -0.48842212557792664,
 -0.39636003971099854,
 0.533961296081543,
 -0.5239564180374146,
 0.9351373314857483,
 0.0822424441576004,
 0.02828170545

In [13]:
t_data.to_csv(output_file, sep = ';')